In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import logging

logging.basicConfig(level = logging.INFO) # Change this to logging.DEBUG pr INFO for more/less detailing debugging information.

# Load in the config files to create iot-ingester actions

In [6]:
from pathlib import Path
from obsproc.core.config_parser import parse_config
from dataclasses import asdict

config_file = Path("config/config.yaml")
config = parse_config(config_file)

# Construct the pipeline from the config
pipeline = {name : getattr(config, name) for name in config.pipeline}
print(f"Global Config:")
for k, v in asdict(config.global_config).items():
    if k == "canonical_variables": v = "..."
    print(f"    {k} : {v}")
    
for name, stage in pipeline.items():
    print(f"\nStage: {name.capitalize()}")
    for pipe in stage:
        print(f"    {str(pipe)}")

Global Config:
    canonical_variables : ...
    config_path : /Users/math/git/iot-ingester-deployment/dockerfiles/worker/iot-ingester/notebooks/sensor.community/config
    data_path : /Users/math/git/iot-ingester-deployment/dockerfiles/worker/iot-ingester/notebooks/sensor.community/data
    code_source : {'repo_status': 'Dirty', 'git_hash': '7b31e2402adb291e13c0d02d989eaf859449567d'}

Stage: Sources
    MultiFileSource(['**/*.csv'], source = 'sensor.community')

Stage: Other_processors
    CSVChunker([Match(state = 'big_file', source = 'sensor.community')])
    QualityControl(a_string)

Stage: Parsers
    CSVParser([Match(state = 'raw', source = 'sensor.community')])

Stage: Aggregators
    TimeAggregator([Match(state = 'parsed', source = 'sensor_community')], 1min, youngest)

Stage: Encoders
    CSVEncoder([Match(state = 'quality_controlled', source = 'sensor.community')])
    ODCEncoder([Match(state = 'quality_controlled', source = 'sensor.community')])


In [11]:
from obsproc.core.bases import FinishMessage, FileMessage, MetaData
from obsproc.aggregators import TimeAggregator

# Pull out the parser object that matches this message
message = FileMessage(
    metadata=MetaData(state='big_file',
        source='sensor_community',
        filepath=Path('data/inputs/sensor_community/2023-08/2023-08_sds011.csv'))) # May need to download this file manually

actions = [s for stage in list(pipeline.values())[1:] for s in stage]

message_history = []
for _ in range(10):
    display(message)
    message_history.append(message)
    matching = [action for action in actions if action.matches(message)]
    
    if not matching: 
        print("No more matches, the message is fully processed!")
        break

    print("That messages matches with this/these action(s): \n\n", "\n".join(str(a) for a in matching))
    action = matching[0]
    print("\nHere's the first output message from passing the previous message to the first matching action: \n")

    # Special case for the TimeAggregator
    if action.__class__.__name__ == "TimeAggregator":
        list(action.process(message)) #need the list call here to pump the iterator to completion even if it doesn't return anything
        message = next(action.process(FinishMessage("We're done!")))
    else:
        message = next(action.process(message))

state,big_file
source,sensor_community
filepath,data/inputs/sensor_community/2023-08/2023-08_sds011.csv


That messages matches with this/these action(s): 

 CSVChunker([Match(state = 'big_file', source = 'sensor.community')])

Here's the first output message from passing the previous message to the first matching action: 



TabularMessage(metadata=MetaData(state='raw', source='sensor_community', observation_variable=None, time_slice=None, encoded_format=None, filepath=None), history=[PreviousActionInfo(action=ActionInfo(name='CSVChunker', code=None), message=MessageInfo(name='FileMessage', metadata=MetaData(state='big_file', source='sensor_community', observation_variable=None, time_slice=None, encoded_format=None, filepath=PosixPath('data/inputs/sensor_community/2023-08/2023-08_sds011.csv'))))], data=      sensor_id sensor_type  location        lat        lon  \
0         78798      SDS011     68146  51.578000   4.778000   
1         13649      SDS011      6898  50.918000   4.688000   
2         24225      SDS011     51462  51.422000   5.514000   
3         32599      SDS011     19277  51.778000   7.912000   
4         37227      SDS011     23123  43.125047  25.682165   
...         ...         ...       ...        ...        ...   
9995      48413      SDS011     34223  50.867400   0.336100   
9996      68714      SDS011     55692  53.356000  10.030000   
9997      21611      SDS011     47005  52.292000  13.452000   
9998      82139      SDS011     71706  51.985802   5.759143   
9999      50182      SDS011     62717  51.461069   5.542965   

                timestamp    P1  durP1  ratioP1    P2  durP2  ratioP2  
0     2023-08-01T00:00:00   NaN    NaN      NaN   NaN    NaN      NaN  
1     2023-08-01T00:00:00  0.00    NaN      NaN  0.00    NaN      NaN  
2     2023-08-01T00:00:00  0.00    NaN      NaN  0.00    NaN      NaN  
3     2023-08-01T00:00:00  0.00    NaN      NaN  0.00    NaN      NaN  
4     2023-08-01T00:00:00  0.00    NaN      NaN  0.00    NaN      NaN  
...                   ...   ...    ...      ...   ...    ...      ...  
9995  2023-08-01T00:02:09  0.00    NaN      NaN  0.00    NaN      NaN  
9996  2023-08-01T00:02:09  0.00    NaN      NaN  0.00    NaN      NaN  
9997  2023-08-01T00:02:09  0.20    NaN      NaN  0.20    NaN      NaN  
9998  2023-08-01T00:02:09  0.50    NaN      NaN  0.30    NaN      NaN  
9999  2023-08-01T00:02:09  0.70    NaN      NaN  0.50    NaN      NaN  

[10000 rows x 12 columns], columns=[])

That messages matches with this/these action(s): 

 CSVParser([Match(state = 'raw', source = 'sensor.community')])

Here's the first output message from passing the previous message to the first matching action: 



TabularMessage(metadata=MetaData(state='parsed', source='sensor_community', observation_variable='P1', time_slice=None, encoded_format=None, filepath=None), history=[PreviousActionInfo(action=ActionInfo(name='CSVChunker', code=None), message=MessageInfo(name='FileMessage', metadata=MetaData(state='big_file', source='sensor_community', observation_variable=None, time_slice=None, encoded_format=None, filepath=PosixPath('data/inputs/sensor_community/2023-08/2023-08_sds011.csv')))), PreviousActionInfo(action=ActionInfo(name='CSVParser', code=None), message=MessageInfo(name='TabularMessage', metadata=MetaData(state='raw', source='sensor_community', observation_variable=None, time_slice=None, encoded_format=None, filepath=None)))], data=                          time station_id sensor_type  location        lat  \
0    2023-08-01 00:00:00+00:00      78798      SDS011     68146  51.578000   
1    2023-08-01 00:00:00+00:00      13649      SDS011      6898  50.918000   
2    2023-08-01 00:00:00+00:00      24225      SDS011     51462  51.422000   
3    2023-08-01 00:00:00+00:00      32599      SDS011     19277  51.778000   
4    2023-08-01 00:00:00+00:00      37227      SDS011     23123  43.125047   
...                        ...        ...         ...       ...        ...   
9995 2023-08-01 00:02:09+00:00      48413      SDS011     34223  50.867400   
9996 2023-08-01 00:02:09+00:00      68714      SDS011     55692  53.356000   
9997 2023-08-01 00:02:09+00:00      21611      SDS011     47005  52.292000   
9998 2023-08-01 00:02:09+00:00      82139      SDS011     71706  51.985802   
9999 2023-08-01 00:02:09+00:00      50182      SDS011     62717  51.461069   

            lon   P1  
0      4.778000  NaN  
1      4.688000  0.0  
2      5.514000  0.0  
3      7.912000  0.0  
4     25.682165  0.0  
...         ...  ...  
9995   0.336100  0.0  
9996  10.030000  0.0  
9997  13.452000  0.2  
9998   5.759143  0.5  
9999   5.542965  0.7  

[10000 rows x 7 columns], columns=[CanonicalVariable(name='time', desc='The time that the observation was made.', unit=None, CRS=None, WMO=False, dtype='datetime64[ns, UTC]', output=False, codetype=None, varno=None, obstype=None), CanonicalVariable(name='station_id', desc='A unique identifer for a stationary sensor.', unit=None, CRS=None, WMO=False, dtype='object', output=False, codetype=None, varno=None, obstype=None), CanonicalVariable(name='sensor_type', desc=None, unit=None, CRS=None, WMO=False, dtype='object', output=False, codetype=None, varno=None, obstype=None), CanonicalVariable(name='location', desc=None, unit=None, CRS=None, WMO=False, dtype='int', output=False, codetype=None, varno=None, obstype=None), CanonicalVariable(name='lat', desc='The lattitude of the observation, referenced to WGS84 (EPSG: 4326)', unit='°', CRS='WGS84', WMO=False, dtype='float64', output=False, codetype=None, varno=None, obstype=None), CanonicalVariable(name='lon', desc='The longitude of the observation, referenced to WGS84 (EPSG: 4326)', unit='°', CRS='WGS84', WMO=False, dtype='float64', output=False, codetype=None, varno=None, obstype=None), CanonicalVariable(name='P1', desc=None, unit=None, CRS=None, WMO=False, dtype='float64', output=True, codetype=-1, varno=-1, obstype=-1)])

...

That messages matches with this/these action(s): 

 QualityControl(a_string)

Here's the first output message from passing the previous message to the first matching action: 



state,quality_controlled
source,sensor_community
observation_variable,P1
time_slice,2023-08-01 00:00
name,FileMessage
metadata,"MetaData(source = sensor_community, variable = None)"
name,CSVChunker
name,TabularMessage
metadata,"MetaData(source = sensor_community, variable = None)"
name,CSVParser
name,TabularMessage


That messages matches with this/these action(s): 

 CSVEncoder([Match(state = 'quality_controlled', source = 'sensor.community')])
ODCEncoder([Match(state = 'quality_controlled', source = 'sensor.community')])

Here's the first output message from passing the previous message to the first matching action: 



state,encoded
source,sensor_community
observation_variable,P1
time_slice,2023-08-01 00:00
encoded_format,csv
filepath,/Users/math/git/iot-ingester-deployment/dockerfiles/worker/iot-ingester/notebooks/sensor.communit...
name,FileMessage
metadata,"MetaData(source = sensor_community, variable = None)"
name,CSVChunker
name,TabularMessage
metadata,"MetaData(source = sensor_community, variable = None)"


No more matches, the message is fully processed!


In [17]:
# Pull out the parser object that matches this message
message = FileMessage(
    metadata=MetaData(state='big_file',
        source='sensor_community',
        filepath=Path('data/inputs/sensor_community/2023-08/2023-08_sds011.csv'))) # May need to download this file manually


message_history = []
for _ in range(4):
    message_history.append(message)
    matching = [action for action in actions if action.matches(message)]
    
    if not matching: 
        break

    action = matching[0]

    # Special case for the TimeAggregator
    if action.__class__.__name__ == "TimeAggregator":
        list(action.process(message)) #need the list call here to pump the iterator to completion even if it doesn't return anything
        message = next(action.process(FinishMessage("We're done!")))
    else:
        message = next(action.process(message))

message

state,quality_controlled
source,sensor_community
observation_variable,P1
time_slice,2023-08-01 00:00
name,FileMessage
metadata,"MetaData(source = sensor_community, variable = None)"
name,CSVChunker
name,TabularMessage
metadata,"MetaData(source = sensor_community, variable = None)"
name,CSVParser
name,TabularMessage
